# Module imports

In [8]:
%pip install pandas matplotlib scikit-learn --quiet
# Install and import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Supress warnings
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Loading the data

In [9]:
# Define the path to the database and the table name (from my private darts-data-tracker repo)
database_url = '../../darts-data-tracker/darts_stats.db'
game_table_name = 'darts_overall_match_stats'

In [10]:
def load_data():
    """
    Load the data from the database and return a pandas dataframe

    Returns:
        df (pd.DataFrame): A pandas dataframe containing the data from the database
    """
    conn = sqlite3.connect(database_url)
    df = pd.read_sql_query(f"SELECT * FROM {game_table_name}", conn)
    conn.close()
    return df

def convert_date_and_time(df):
    """
    Convert date and time columns into a combined datetime column, replace original columns, and sort.
    
    Args:
        df (pd.DataFrame): Input dataframe with separate date and time columns
        
    Returns:
        pd.DataFrame: Modified dataframe with combined datetime column in original date position
    """
    # Convert string dates to proper datetime objects
    df['date'] = pd.to_datetime(df['date'], format='%d %b, %Y')
    
    # Clean time values by extracting just hours:minutes
    df['time'] = df['time'].str.extract(r'(\d{2}:\d{2})', expand=False)
    
    # Merge date and time into single datetime column
    df['datetime'] = pd.to_datetime(
        df['date'].dt.strftime('%Y-%m-%d') + ' ' + df['time'],
        format='%Y-%m-%d %H:%M'
    )
    
    # Record original position of date column for later insertion
    original_column_position = df.columns.get_loc('date')
    
    # Remove obsolete columns
    df = df.drop(['date', 'time'], axis=1)
    
    # Reorganize columns to place datetime where date was originally
    column_list = df.columns.tolist()
    column_list.remove('datetime')  
    column_list.insert(original_column_position, 'datetime')  
    df = df[column_list]
    
    # Sort chronologically and reset index
    df = df.sort_values(by='datetime').reset_index(drop=True)
    
    return df

# Load the data
darts_matches = load_data()

# Apply the conversion and sorting
darts_matches = convert_date_and_time(darts_matches)

# Data Preprocessing

In [11]:
def drop_unnecessary_columns(df):
    """
    Drop unnecessary columns from the dataframe

    Args:
        df (pd.DataFrame): The dataframe to drop columns from

    Returns:
        pd.DataFrame: The dataframe with the specified columns dropped
    """
    try:
        df = df.drop(['uid', 'event_title', 'leg_count'], axis=1)
    except KeyError:
        pass
    return df

def create_dual_perspective_df(df):
    """
    Transforms match data to include both player perspectives per match.
    
    Creates two rows per match:
    1. Home team as player, away team as opponent
    2. Away team as player, home team as opponent
    
    Args:
        df (pd.DataFrame): Original dataframe with home/away columns
    
    Returns:
        pd.DataFrame: Restructured dataframe with player/opponent perspectives
    """
    
    # Create home team perspective
    home = df.rename(columns=lambda x: x.replace('home_team', 'player').replace('away_team', 'opponent'))
    
    # Create away team perspective with column swaps
    away = df.rename(columns=lambda x: x.replace('away_team', 'player').replace('home_team', 'opponent'))
    
    # Combine both perspectives
    dual_df = pd.concat([home, away], ignore_index=True)
    
    # Create target variable
    dual_df['player_won'] = (dual_df['player_legs_won'] > dual_df['opponent_legs_won']).astype(int)
    
    # Clean column order
    base_cols = ['datetime']
    player_cols = [c for c in dual_df if c.startswith('player')]
    opponent_cols = [c for c in dual_df if c.startswith('opponent')]
    
    df = dual_df[base_cols + player_cols + opponent_cols]
    df.sort_values(by='datetime', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def manage_checkout_columns(df):
    """
    Split checkouts column into attempts and successful checkouts. 
    Drop checkouts of 100+ from the dataframe.
    Handles invalid checkout formats and zero denominators.
    Organizes checkout columns at end of player/opponent sections.
    Moves target variable to end and renames it.

    Args:
        df (pd.DataFrame): The dataframe to process

    Returns:
        pd.DataFrame: Processed dataframe with organized columns
    """
    # Remove 100+ checkout columns
    df = df.drop(['player_checkouts_100_plus', 'opponent_checkouts_100_plus'], axis=1)

    def split_checkout(col):
        """Split checkout column into made/attempted with error handling"""
        result = col.str.split('/', expand=True)
        return (
            result[0].fillna('0').astype(int),  # Made
            result[1].fillna('0').astype(int)   # Attempted
        )

    # Process player checkouts
    player_made, player_attempted = split_checkout(df['player_checkouts'])
    df = df.assign(
        player_checkouts_made=player_made,
        player_checkouts_attempted=player_attempted,
        player_checkout_percentage=np.where(
            player_attempted > 0,
            player_made / player_attempted * 100,
            0.0
        )
    )

    # Process opponent checkouts
    opponent_made, opponent_attempted = split_checkout(df['opponent_checkouts'])
    df = df.assign(
        opponent_checkouts_made=opponent_made,
        opponent_checkouts_attempted=opponent_attempted,
        opponent_checkout_percentage=np.where(
            opponent_attempted > 0,
            opponent_made / opponent_attempted * 100,
            0.0
        )
    )

    # Remove original checkout columns
    df = df.drop(['player_checkouts', 'opponent_checkouts'], axis=1)
    
    # Rename target variable
    df = df.rename(columns={'player_won': 'did_player_win'})

    # Reorganize column order
    cols = df.columns.tolist()
    
    # Columns to reposition
    player_cols = ['player_checkouts_made', 'player_checkouts_attempted']
    opponent_cols = ['opponent_checkouts_made', 'opponent_checkouts_attempted']
    target_col = 'did_player_win'
    
    # Remove from current positions
    cols = [c for c in cols if c not in player_cols + opponent_cols + [target_col]]
    
    # Insert player checkout columns after player_checkout_percentage
    player_cp_idx = cols.index('player_checkout_percentage')
    cols[player_cp_idx+1:player_cp_idx+1] = player_cols
    
    # Insert opponent checkout columns after opponent_checkout_percentage
    opponent_cp_idx = cols.index('opponent_checkout_percentage')
    cols[opponent_cp_idx+1:opponent_cp_idx+1] = opponent_cols
    
    # Add target column at end
    cols.append(target_col)
    
    return df[cols]

def convert_legs_won_columns_to_int(df):
    """
    Convert legs_won columns to int

    Args:
        df (pd.DataFrame): The dataframe to convert

    Returns:
        pd.DataFrame: The converted dataframe
    """
    df['player_legs_won'] = df['player_legs_won'].astype(int)
    df['opponent_legs_won'] = df['opponent_legs_won'].astype(int)
    return df

darts_matches = drop_unnecessary_columns(darts_matches)
darts_matches = create_dual_perspective_df(darts_matches)
darts_matches = manage_checkout_columns(darts_matches)
darts_matches = convert_legs_won_columns_to_int(darts_matches)

In [12]:
# Print columns and their types
for column in darts_matches.columns:
    print(f"{column}: {darts_matches[column].dtype}")

datetime: datetime64[ns]
player_name: object
player_legs_won: int64
player_average: float64
player_100_plus_thrown: int64
player_140_plus_thrown: int64
player_180_thrown: int64
player_highest_checkout: int64
player_checkout_percentage: float64
player_checkouts_made: int64
player_checkouts_attempted: int64
opponent_name: object
opponent_legs_won: int64
opponent_average: float64
opponent_100_plus_thrown: int64
opponent_140_plus_thrown: int64
opponent_180_thrown: int64
opponent_highest_checkout: int64
opponent_checkout_percentage: float64
opponent_checkouts_made: int64
opponent_checkouts_attempted: int64
did_player_win: int64


# Feature Engineering

In [13]:
def drop_opponent_columns(df):
    """
    Drop any columns that start with 'opponent_' except for 'opponent_name'.

    Args:
        df (pd.DataFrame): The dataframe from which to drop columns.

    Returns:
        pd.DataFrame: The dataframe with specified opponent columns dropped.
    """
    return df.drop(columns=[col for col in df.columns if col.startswith('opponent_') and col != 'opponent_name'])

# Preprocessing
darts_matches = drop_opponent_columns(darts_matches)

# Print the shape of the dataframe where player_checkouts_attempted is 0 but player_checkouts_made is not 0
temp_more_checkouts_than_attempts = darts_matches[
    (darts_matches['player_checkouts_attempted'] < darts_matches['player_checkouts_made'])
]
print(f'Records where checkouts are not tracked properly: {temp_more_checkouts_than_attempts.shape}')

# Display all rows where player is 'Dom Taylor' and checkouts are not tracked properly
display(darts_matches[
    (darts_matches['player_name'] == 'Dom Taylor') &
    (darts_matches['player_checkouts_attempted'] < darts_matches['player_checkouts_made'])
])

Records where checkouts are not tracked properly: (635, 13)


,datetime,player_name,player_legs_won,player_average,player_100_plus_thrown,player_140_plus_thrown,player_180_thrown,player_highest_checkout,player_checkout_percentage,player_checkouts_made,player_checkouts_attempted,opponent_name,did_player_win
132,2024-02-11 13:05:00,Dom Taylor,6,93.82,10,5,2,76,0.0,6,0,Stephen Bunting,1
396,2024-02-19 13:50:00,Dom Taylor,2,96.97,7,8,1,64,0.0,2,0,Michael van Gerwen,0
1031,2024-04-07 14:10:00,Dom Taylor,6,95.13,11,7,2,80,0.0,6,0,Geert Nentjes,1
2083,2024-06-11 15:35:00,Dom Taylor,3,92.66,9,7,0,60,0.0,3,0,D. Van den Bergh,0
2407,2024-07-02 13:05:00,Dom Taylor,6,100.76,16,7,1,106,0.0,6,0,Dirk van Duijvenbode,1
3628,2024-10-14 13:20:00,Dom Taylor,6,98.06,13,6,0,154,0.0,6,0,Stephen Burton,1
3943,2024-10-29 13:25:00,Dom Taylor,4,85.12,11,5,0,120,0.0,4,0,William Borland,0


In [14]:
def regression_impute_checkout_attempts(df, random_state=None):
    """
    Fixes invalid checkout attempts using regression on valid patterns
    (More appropriate than logistic regression for continuous outcomes)

    Args:
        df (pd.DataFrame): The dataframe to process
        random_state (int): The random state for reproducibility

    Returns:
        pd.DataFrame: The processed dataframe with invalid checkout attempts fixed
    """
    df = df.copy()
    
    # Identify invalid cases
    invalid_mask = (df['player_checkouts_attempted'] < df['player_checkouts_made']) | \
                   ((df['player_checkouts_attempted'] == 0) & (df['player_checkouts_made'] > 0))
    
    if not invalid_mask.any():
        print("No invalid checkout attempts found")
        return df

    # Prepare valid/invalid data splits
    valid_data = df[~invalid_mask].dropna(subset=['player_checkouts_attempted'])
    invalid_data = df[invalid_mask]
    
    if valid_data.empty or invalid_data.empty:
        return df

    # Feature engineering
    features = [
        'player_checkouts_made',
        'player_legs_won', 
        'player_average',
        'player_100_plus_thrown',
        'player_140_plus_thrown',
        'player_180_thrown',
        'player_highest_checkout',
        'did_player_win'
    ]
    
    # Regression pipeline with regularization
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge(alpha=1.0))  # Regularized regression
    ])
    
    # Train on valid data
    model.fit(valid_data[features], valid_data['player_checkouts_attempted'])
    
    # Predict attempts for invalid cases
    pred_attempts = model.predict(invalid_data[features])
    pred_attempts = np.ceil(pred_attempts).astype(int)  # Round up to whole numbers
    
    # Apply logical constraints
    min_attempts = np.where(
        invalid_data['player_checkouts_made'] > 0,
        invalid_data['player_checkouts_made'] + 1,  # At least made + 1
        1  # Minimum 1 attempt
    )
    final_attempts = np.maximum(pred_attempts, min_attempts)
    
    # Update dataframe
    df.loc[invalid_mask, 'player_checkouts_attempted'] = final_attempts
    
    # Final validation and cleanup
    df['player_checkout_percentage'] = np.where(
        df['player_checkouts_attempted'] > 0,
        (df['player_checkouts_made'] / df['player_checkouts_attempted']) * 100,
        0
    ).clip(0, 100).round(1)
    
    # Ensure no remaining invalid cases
    final_check = df[df['player_checkouts_attempted'] < df['player_checkouts_made']]
    if not final_check.empty:
        df.loc[final_check.index, 'player_checkouts_attempted'] = \
            final_check['player_checkouts_made'] + 1
    
    print(f"Corrected {invalid_mask.sum()} invalid rows")
    return df

darts_matches = regression_impute_checkout_attempts(darts_matches)

# Display all rows where player is 'Dom Taylor'
display(darts_matches[darts_matches['player_name'] == 'Dom Taylor'])

Corrected 635 invalid rows
Records where checkouts are not tracked properly: (0, 13)


,datetime,player_name,player_legs_won,player_average,player_100_plus_thrown,player_140_plus_thrown,player_180_thrown,player_highest_checkout,player_checkout_percentage,player_checkouts_made,player_checkouts_attempted,opponent_name,did_player_win
132,2024-02-11 13:05:00,Dom Taylor,6,93.82,10,5,2,76,40.0,6,15.0,Stephen Bunting,1
396,2024-02-19 13:50:00,Dom Taylor,2,96.97,7,8,1,64,40.0,2,5.0,Michael van Gerwen,0
1031,2024-04-07 14:10:00,Dom Taylor,6,95.13,11,7,2,80,40.0,6,15.0,Geert Nentjes,1
1403,2024-04-25 15:40:00,Dom Taylor,6,100.20,8,6,1,64,54.5,6,11.0,Christian Goedl,1
1421,2024-04-26 12:00:00,Dom Taylor,2,91.02,14,4,1,123,50.0,2,4.0,Krzysztof Ratajski,0
1777,2024-05-23 12:00:00,Dom Taylor,6,102.26,9,12,1,64,37.5,6,16.0,Jose de Sousa,1
1838,2024-05-24 21:30:00,Dom Taylor,5,92.35,12,8,2,38,35.7,5,14.0,Gian van Veen,0
2083,2024-06-11 15:35:00,Dom Taylor,3,92.66,9,7,0,60,37.5,3,8.0,D. Van den Bergh,0
2130,2024-06-20 12:00:00,Dom Taylor,6,97.15,11,4,1,60,54.5,6,11.0,Radek Szaganski,1
2165,2024-06-21 13:00:00,Dom Taylor,4,97.37,21,2,2,106,50.0,4,8.0,Damon Heta,0


In [146]:
def get_historical_matches(df, current_date, player_name):
    """
    Get all matches for a player before the current date, sorted chronologically.

    Args:
        df (pd.DataFrame): The dataframe containing match data.
        current_date (datetime): The date to filter matches before.
        player_name (str): The name of the player for whom to retrieve matches.

    Returns:
        pd.DataFrame: A dataframe containing historical matches for the specified player.
    """
    return df[
        (df['player_name'] == player_name) &
        (df['datetime'] < current_date)
    ].sort_values('datetime')

def calculate_averages(historical_df, base_features, n_windows=[6], min_matches=1):
    """
    Calculate averages ensuring no NaN propagation.

    Args:
        historical_df (pd.DataFrame): The dataframe containing historical match data.
        base_features (list): List of features for which to calculate averages.
        n_windows (list): List of window sizes for rolling averages.
        min_matches (int): Minimum number of matches required to calculate rolling averages.

    Returns:
        dict: A dictionary containing calculated averages.
    """
    stats = {}
    
    # Career averages (all available history)
    if len(historical_df) >= 1:  # Need at least 1 match
        for feat in base_features:
            stats[f'career_avg_{feat}'] = historical_df[feat].mean()
    else:
        for feat in base_features:
            stats[f'career_avg_{feat}'] = None
    
    # Rolling averages
    for n in n_windows:
        window_df = historical_df.tail(n)
        if len(window_df) >= min_matches:
            for feat in base_features:
                stats[f'rolling_{n}_avg_{feat}'] = window_df[feat].mean()
        else:
            for feat in base_features:
                stats[f'rolling_{n}_avg_{feat}'] = None
    
    return stats

def append_stats(df, base_features, n_windows=[6], min_matches=1):
    """
    Perform row-by-row calculation that never references new columns.

    Args:
        df (pd.DataFrame): The dataframe to which stats will be appended.
        base_features (list): List of features for which to calculate averages.
        n_windows (list): List of window sizes for rolling averages.
        min_matches (int): Minimum number of matches required to calculate rolling averages.

    Returns:
        pd.DataFrame: The dataframe with appended statistics.
    """
    # Create copy to avoid modifying original dataframe
    result_df = df.copy()
    
    # Pre-initialize all stat columns with None
    stat_columns = []
    for feat in base_features:
        result_df[f'career_avg_{feat}'] = None
        stat_columns.append(f'career_avg_{feat}')
    for n in n_windows:
        for feat in base_features:
            col_name = f'rolling_{n}_avg_{feat}'
            result_df[col_name] = None
            stat_columns.append(col_name)
    
    # Process each row
    total = len(result_df)
    for idx, row in result_df.iterrows():
        # Get historical data using ORIGINAL dataframe
        historical = get_historical_matches(
            df,  # Use original DF without any stat columns
            row['datetime'],
            row['player_name']
        )
        
        # Calculate stats
        stats = calculate_averages(historical, base_features, n_windows, min_matches)
        
        # Update only the stat columns in result_df
        for col in stat_columns:
            result_df.at[idx, col] = stats.get(col, None)
        
        # Progress tracking
        if idx % 10 == 0:
            print(f'\rProcessed {idx+1}/{total} rows...', end=' ')
    return result_df

# Legs won is not included as it could be deceptive given match formats with different numbers of legs
BASE_FEATURES = [
    'player_average', 'player_100_plus_thrown',
    'player_140_plus_thrown', 'player_180_thrown', 'player_highest_checkout',
    'player_checkout_percentage', 'player_checkouts_made', 'player_checkouts_attempted'
]

# Apply the calculation
darts_matches = append_stats(
    df=darts_matches,
    base_features=BASE_FEATURES,
    n_windows=[6], 
    min_matches=1   
)

# Print columns after processing
print("\nColumns after processing:", darts_matches.columns.tolist())

Processed 4551/4556 rows...   
Columns after processing: ['datetime', 'player_name', 'player_legs_won', 'player_average', 'player_100_plus_thrown', 'player_140_plus_thrown', 'player_180_thrown', 'player_highest_checkout', 'player_checkout_percentage', 'player_checkouts_made', 'player_checkouts_attempted', 'opponent_name', 'did_player_win', 'career_avg_player_average', 'career_avg_player_100_plus_thrown', 'career_avg_player_140_plus_thrown', 'career_avg_player_180_thrown', 'career_avg_player_highest_checkout', 'career_avg_player_checkout_percentage', 'career_avg_player_checkouts_made', 'career_avg_player_checkouts_attempted', 'rolling_6_avg_player_average', 'rolling_6_avg_player_100_plus_thrown', 'rolling_6_avg_player_140_plus_thrown', 'rolling_6_avg_player_180_thrown', 'rolling_6_avg_player_highest_checkout', 'rolling_6_avg_player_checkout_percentage', 'rolling_6_avg_player_checkouts_made', 'rolling_6_avg_player_checkouts_attempted']


In [147]:
def add_h2h_record(df):
    """
    Adds historical head-to-head (H2H) records between players based on previous matches.
    Operates on the dual perspective dataframe to maintain match symmetry.
    
    Args:
        df (pd.DataFrame): Dual perspective dataframe with 'player_name', 'opponent_name', 
                          'did_player_win', and 'datetime' columns
                          
    Returns:
        pd.DataFrame: Original dataframe with added 'previous_h2h_wins' and 'previous_h2h_losses' columns
    """
    
    # Create working copy to preserve original data
    temp_df = df.copy()
    
    # Sort matches chronologically for accurate cumulative counts
    temp_df = temp_df.sort_values('datetime').reset_index(drop=True)
    
    # Track original index for final reordering
    temp_df['original_index'] = temp_df.index
    
    # Calculate cumulative match counts and wins for each player-opponent pair
    temp_df['cum_matches'] = temp_df.groupby(['player_name', 'opponent_name']).cumcount() + 1
    temp_df['cum_wins'] = temp_df.groupby(['player_name', 'opponent_name'])['did_player_win'].cumsum()
    
    # Shift results to exclude current match from H2H counts
    temp_df['previous_h2h_wins'] = temp_df.groupby(['player_name', 'opponent_name'])['cum_wins'].shift(1).fillna(0).astype(int)
    temp_df['previous_h2h_losses'] = (
        temp_df.groupby(['player_name', 'opponent_name'])['cum_matches'].shift(1).fillna(0) 
        - temp_df['previous_h2h_wins']
    ).astype(int)
    
    # Clean up temporary columns
    temp_df = temp_df.drop(['cum_matches', 'cum_wins', 'original_index'], axis=1)

    # Calculate the difference between wins and losses
    temp_df['previous_h2h_differential'] = temp_df['previous_h2h_wins'] - temp_df['previous_h2h_losses']
    
    # Reorder to match original input sequence
    return temp_df.sort_index()

darts_matches = add_h2h_record(darts_matches)
display(darts_matches)

,datetime,player_name,player_legs_won,player_average,player_100_plus_thrown,player_140_plus_thrown,player_180_thrown,player_highest_checkout,player_checkout_percentage,player_checkouts_made,...,rolling_6_avg_player_100_plus_thrown,rolling_6_avg_player_140_plus_thrown,rolling_6_avg_player_180_thrown,rolling_6_avg_player_highest_checkout,rolling_6_avg_player_checkout_percentage,rolling_6_avg_player_checkouts_made,rolling_6_avg_player_checkouts_attempted,previous_h2h_wins,previous_h2h_losses,previous_h2h_differential
0,2024-01-17 16:10:00,Peter Wright,6,91.69,17,8,1,40,46.153846,6,...,None,None,None,None,None,None,None,0,0,0
1,2024-01-17 16:10:00,Haruki Muramatsu,5,87.11,15,4,2,170,35.714286,5,...,None,None,None,None,None,None,None,0,0,0
2,2024-01-17 16:50:00,Gerwyn Price,6,92.13,8,7,3,93,28.571429,6,...,None,None,None,None,None,None,None,0,0,0
3,2024-01-17 16:50:00,Reynaldo Rivera,4,94.86,16,11,0,104,33.333333,4,...,None,None,None,None,None,None,None,0,0,0
4,2024-01-17 17:30:00,Nathan Aspinall,6,97.20,14,8,2,102,28.571429,6,...,None,None,None,None,None,None,None,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,2025-01-16 18:10:00,Luke Humphries,6,94.76,19,6,5,123,50.000000,6,...,19.333333,10.333333,3.5,143.833333,53.807832,8.833333,18.833333,4,3,1
4552,2025-01-16 18:50:00,Peter Wright,4,91.15,16,6,4,36,18.181818,4,...,23.166667,11.666667,4.5,113.5,44.983592,9.833333,22.5,2,4,-2
4553,2025-01-16 18:50:00,Gerwyn Price,7,92.58,13,4,3,124,38.888889,7,...,24.166667,11.5,6.166667,112.833333,37.440115,11.166667,33.666667,4,2,2
4554,2025-01-16 19:30:00,Stephen Bunting,8,99.33,13,8,5,85,47.058824,8,...,19.833333,13.833333,4.833333,111.0,50.997698,9.833333,21.833333,1,2,-1
